In [ ]:
include("emulation_functions.jl")
using LinearAlgebra
using Random
using NetCDF
using NCDatasets
using Dates
using LaTeXStrings
using BenchmarkTools
using Suppressor
using StatsBase
using Turing
using ScikitLearn
using CSV
using DataFrames
using JLD2

### train GP emulator for the 4 parameter 200 ensemble ELM simulation

In [ ]:
load_from_file = false

In [ ]:
# import data
fname_params = "../data/sobol_sequence_ensemble_4par_200_QuasiMonteCarlo_0.txt"
fname_GPP = "../data/US_Me2_GPP_g0001_g0200_2003_2007_res05_4par.nc"
GPP =ncread(fname_GPP, "GPP");
time = ncread(fname_GPP, "time")
ELM_Me2 = Dataset("../data/US_Me2_GPP_g0001_g0200_2003_2007_res05_4par.nc")
t = ELM_Me2["time"]
time1=year.(t)+dayofyear.(t)/365
GPP1 = dropdims(GPP,dims=1)
id=findall(.!all.(x -> x .== 0, eachcol(GPP1)));

In [ ]:
# Load param vals, remove 2 missing columns at end
df = CSV.read(fname_params, DataFrame, header=false)
s = Matrix(df);
df = df[!, map(x->!all(ismissing, x), eachcol(df))]
df = Matrix(DataFrame([[names(df)]; collect.(eachrow(df))], [:column; Symbol.(axes(df, 1))]))[:,2:end]

Separate test and train data

In [ ]:
# Set a seed for reproducibility
Random.seed!(11);
indices_train0 = StatsBase.sample(1:200, 160, replace=false)
indices_test0 = [i for i in 1:200 if i ∉ indices_train0]
indices_train=intersect(indices_train0, id)
indices_test=intersect(indices_test0, id)
scatter(s[indices_test ,1], s[indices_test ,2], xlabel="p1", ylabel="p2", title="40 testing set", legend=false, grid=false, framestyle=:box)

In [ ]:
# dimensions: N_time points X N_observations
y_train = Float64.(GPP[1,:,indices_train])
p_train = Matrix(Float64.(df[:, indices_train]))
p_train_norm = p_train # Using normalized parameter values for training 

y_test = Float64.(GPP[1,:,indices_test])
p_test = Matrix(Float64.(df[:, indices_test]));

y_all = Float64.(GPP[1,:,:])
p_all = Matrix(Float64.(df[:, :]));

Define covariance and mean functions for GP

In [ ]:
# Functions for defining inputs to GP Training
# Select mean and covariance function
mZero = GaussianProcesses.MeanZero() # Zero mean function
kern = GaussianProcesses.SEArd(zeros(size(p_train, 1)), 0.0)

In [ ]:
@time if !load_from_file
    # For PCA method, each column of y_train is an observation 
    pca_fit= MultivariateStats.fit(PCA, y_train; pratio=1.0, maxoutdim=12) # perform PCA on X, retaining first K PCs
    Z_train = project(pca_fit, y_train)
    Z_test = project(pca_fit, y_test)

    μ_z = mean(Z_train, dims=2)
    σ_z = std(Z_train, dims=2)

    Z_train_std = (Z_train .- μ_z) ./ σ_z # Standardize the latent variables

    array_GP = Array{GPE}(undef,size(pca_fit, 2)) # Create an empty matrix
    for j = 1: size(pca_fit, 2)     
        gp = GaussianProcesses.GP(p_train_norm, Z_train_std[j, :], mZero, kern)
        optimize!(gp, domean = true, kern = true, noise = false, f_tol = 1e-40, g_tol = 1e-14, iterations = 2000)   # Optimizat parameters
        array_GP[j] = deepcopy(gp)   # Save each learned GP model to array_GP via deepcopy.
    end

    p_emulator = (GPs = array_GP, T_PCA = pca_fit, μ_z = μ_z, σ_z = σ_z);
    jldsave("emulator_GPP_US_Me2_200_4par.jld2"; array_GP, pca_fit, μ_z, σ_z)
else
    @load "emulator_GPP_US_Me2_200_4par.jld2" array_GP pca_fit μ_z σ_z
    p_emulator = (GPs = array_GP, T_PCA = pca_fit, μ_z = μ_z, σ_z = σ_z);
end

